In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tools import BaseK,show_plot,Solver,Differ
from matplotlib.ticker import FuncFormatter
from matplotlib.widgets import Slider, Button
import copy

In [2]:
%matplotlib qt

In [3]:
class K(BaseK):
    light = 0.0001
    Ket = 1e9
    Ke = 1e9
    Kee = 0

    KH = 1e9
    KHH = 0

    Kr = 1e9
    Kp = 0.001
    KqH = 2000
    Kdisp = 1e9
    Kph = 1e-5
    Ks = 2
    Kd = 0.05
    Kc = 1
    KrD = 1e9

In [4]:
T = np.linspace(0, 0.000_1, 100)

In [5]:
def system_full(t, C, k=K()):
    [Q, DH, QHH, D, Qt, QHD, QH, QD, Q_DH] = C

    ret = k.Ket*k.light * Q
    ree = k.Kee * Qt
    re = k.Ke * Qt * DH
    rqH = k.KqH * Qt * QHH

    rph = k.Kph * Qt

    rs = k.Ks * Qt * QHD
    rc = k.Kc * Qt * D

    rHH = k.KHH * Q_DH
    rH = k.KH * Q_DH

    rc = k.Kr * QH * D
    rp = k.Kp * QHD
    rdisp = k.Kdisp * QH * QH
    rd = k.Kd * QD
    rrD = k.KrD * D * D

    Q = -ret + ree + rdisp + rd
    Qt = ret - ree - re - rqH - rph - rs - rc + rHH
    DH = -re + rHH
    Q_DH = re - rHH - rH
    QHH = -rqH + rp + rdisp
    QH = 2 * rqH + rs + rH - rc - 2 * rdisp
    QHD = -rs + rc - rp
    QD = rs + rc - rd
    D = -rc + rH - rc + rd - 2 * rrD
    return [Q, DH, QHH, D, Qt, QHD, QH, QD, Q_DH]


c_full = '[Q, DH, QHH, D, Qt, QHD, QH, QD, Q_DH]'
initial_full = [1, 1, 0, 0, 0, 0, 0, 0, 0]

In [6]:
def system_ions(t, C, k=K()):
    [Q, DH, QHH, D, QHD, QH, QD, Qm, DHp] = C
    re = k.Ke * k.light * Q * DH
    rH = k.KH * Qm * DHp
    rr = k.Kr * QH * D
    rp = k.Kp * QHD
    rqH = k.KqH * Q * QHH
    rdisp = k.Kdisp * QH * QH
    rph = k.Kph * Q
    rs = k.Ks * Q * QHD
    rd = k.Kd * QD
    rc = k.Kc * Q * D
    rrD = k.KrD * D * D

    Q = -re - rqH - rph - rs - rc + rdisp + rd
    DH = -re
    QHH = -rqH + rp + rdisp
    D = -rr - rc - 2 * rrD + rH + rd
    QHD = -rp - rs + rr
    QH = -rr - 2 * rdisp + rH + 2 * rqH + rs
    QD = -rd + rs + rc
    Qm = -rH + re
    DHp = -rH + re

    return [Q, DH, QHH, D, QHD, QH, QD, Qm, DHp]


c_ions = '[Q, DH, QHH, D, QHD, QH, QD, Qm, DHp]'
initial_ions = [1, 1, 0, 0, 0, 0, 0, 0, 0]

In [7]:
10**-np.inf

0.0

#   Plot

In [8]:
# y= Solver(system_ions, K(), initial_ions, c_ions, T)
# y = Solver(system_full, K(), initial_full, c_full, T)
Y = Differ(
    Solver(system_full, K(), initial_full, c_full, T),
    Solver(system_ions, K(), initial_ions, c_ions, T),
)
SET_LIM = False
AUTO = False

In [12]:
def foo():
    if True:
        return None
    print(1)

In [13]:
foo()

In [15]:
# Plots
lims = 50
fig, ax = plt.subplots()
gs = plt.GridSpec(2, 2, figure=fig)
fig.subplots_adjust(left=0.25, right=0.99, bottom=0.2, top=0.95, hspace=0.1, wspace=0.1)
ax.xaxis.set_ticklabels([])
ax.yaxis.set_ticklabels([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

# Plots
scal = FuncFormatter(lambda x, pos: f'{x*1000: .2f}')
K_base = Y.K
Y.solve()

plots = {}

base_ax = plt.subplot(gs[0, 0:2])
for c in ['Q', 'DH', 'QHH']:
    plots[c] = base_ax.plot(Y.T, Y[c], label=c)[0]
base_ax.legend()
base_ax.xaxis.set_major_formatter(scal)
if SET_LIM:
    base_ax.set_ylim([-lims, lims])

d_ax = plt.subplot(gs[1, 0])
plots['D'] = d_ax.plot(Y.T, Y['D'], label='D')[0]
d_ax.legend()
d_ax.xaxis.set_major_formatter(scal)
if SET_LIM:
    d_ax.set_ylim([-lims, lims])

other_ax = plt.subplot(gs[1, 1])
other_ax.xaxis.set_major_formatter(scal)
for c in ['QH', 'QHD']:
    plots[c] = other_ax.plot(Y.T, Y[c], label=c)[0]
other_ax.legend()
other_ax.xaxis.set_major_formatter(scal)
if SET_LIM:
    other_ax.set_ylim([-lims, lims])

# Sliders
sliders = {}
K_new = copy.deepcopy(K_base)


def get_slider_action(k):
    def update(val):
        K_new[k] = K_base[k] * 10 ** val
        if not AUTO:
            return None

        Y.solve(K=K_new)
        for c, plot in plots.items():
            plot.set_ydata(Y[c])
        fig.canvas.draw_idle()

    return update


def get_buttom_action(k):
    def update(event):
        K_new[k] = 0
        if not AUTO:
            return None
        Y.solve(K=K_new)
        for c, plot in plots.items():
            plot.set_ydata(Y[c])
        fig.canvas.draw_idle()

    return update


def solve_action(event):
    # for k in K_base:
    #     K_new[k] = K_base[k] * 10 ** sliders[k].val
    Y.solve(K=K_new)
    for c, plot in plots.items():
        plot.set_ydata(Y[c])
    fig.canvas.draw_idle()


but_axes = fig.add_axes([0.05, 0.05, 0.15, 0.03])
solve_button = Button(but_axes, 'Solve', hovercolor='0.975')
solve_button.on_clicked(solve_action)

for i, k in enumerate(K_base):
    k_axes = fig.add_axes(
        [
            0.03, # Left
            0.95 - 0.05 * i, #Top
            0.1, # Right
            0.05, # Wide
        ]
    )
    amp_slider = Slider(
        ax=k_axes,
        label=k,
        valmin=-3,
        valmax=3,
        valinit=0,
        valstep=0.1,
        orientation="horizontal",
    )
    amp_slider.on_changed(get_slider_action(k))
    sliders[k] = amp_slider

    # k_zero_axes = fig.add_axes(
    #     [
    #         0.11, # Left
    #         0.95 - 0.05 * i, #Top
    #         0.15, # Right
    #         0.05, # Wide
    #     ]
    # )
    # zero_button = Button(k_zero_axes, '0', hovercolor='0.975')
    # zero_button.on_clicked(get_buttom_action(k))


In [ ]:
def system_activated(t, C, k=K):
    [Qt,Qs, DH, QHH, D, QHD, QH, QD] = C
    Qs = -R1
    Qt = R1 - R2 - R3 - R4 - R5
    DH = -R2
    QH = R2 + 2*R3 + R4 - R6 - 2*R7
    D = R2 - R5 - R6 + R8 - 2*R10
    QHH = -R3 + R7 + R9
    QHD = -R4 + R6 - R9
    QD = R4 + R5 - R8
    Q = R7 + R8 - R11

    re = k['Ke'] * k['light'] * Q * DH
    rr = k['Kr'] * QH * D
    rp = k['Kp'] * QHD
    rqH = k['KqH'] * Q * QHH
    rdisp = k['Kdisp'] * QH * QH
    rph = k['Kph'] * Q
    rs = k['Ks'] * Q * QHD
    rd = k['Kd'] * QD
    rc = k['Kc'] * Q * D
    rrD = k['KrD'] * D * D

    R1 = k['e']*k['light']*Qs
    R2 = k['Ke']*Qt*DH
    R3 = k['KqH']*Q*QHH
    R4 = k['Ks']*Qt*QHD
    R5 = k['Kr']*Qt*D
    R6 = k['Kr']*QH*D
    R7 = k['Kdisp']*QH*QH
    R8 = k['Kd']*QD
    R9 = k['Kp']*QHD
    R10 = k['KrD']*D*D

    return [Q, DH, QHH, D, QHD, QH, QD]